In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2023
# to_year = datetime.now().year
to_year = 2024
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)


# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2023 to 2024
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading dat

In [4]:
rides

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107
...,...,...
77797681,2024-12-31 23:14:53,145
77797682,2024-12-31 23:04:32,37
77797683,2024-12-31 23:03:16,181
77797684,2024-12-31 23:15:33,165


In [5]:
rides.shape

(77797686, 2)

In [6]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77797686 entries, 0 to 77797685
Data columns (total 2 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_datetime     datetime64[us]
 1   pickup_location_id  int64         
dtypes: datetime64[us](1), int64(1)
memory usage: 1.2 GB


In [7]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [8]:
ts_data.shape

(4561440, 3)

In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4561440 entries, 0 to 4561439
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 52.2 MB


In [10]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
4561435,2024-12-31 19:00:00,263,148
4561436,2024-12-31 20:00:00,263,231
4561437,2024-12-31 21:00:00,263,247
4561438,2024-12-31 22:00:00,263,193


In [12]:
ts_data[ts_data["pickup_hour"] == "2023-03-12 02:00:00"]

,pickup_hour,pickup_location_id,rides
1682,2023-03-12 02:00:00,2,0
19226,2023-03-12 02:00:00,3,0
36770,2023-03-12 02:00:00,4,0
54314,2023-03-12 02:00:00,5,0
71858,2023-03-12 02:00:00,6,0
...,...,...,...
4475402,2023-03-12 02:00:00,259,0
4492946,2023-03-12 02:00:00,260,0
4510490,2023-03-12 02:00:00,261,0
4528034,2023-03-12 02:00:00,262,0


In [14]:
import pytz

est = pytz.timezone('US/Eastern')

ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(est, ambiguous="NaT", nonexistent="NaT")
ts_data = ts_data.dropna(subset=["pickup_hour"])
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00-05:00,2,0
1,2023-01-01 01:00:00-05:00,2,0
2,2023-01-01 02:00:00-05:00,2,0
3,2023-01-01 03:00:00-05:00,2,0
4,2023-01-01 04:00:00-05:00,2,0
...,...,...,...
4561435,2024-12-31 19:00:00-05:00,263,148
4561436,2024-12-31 20:00:00-05:00,263,231
4561437,2024-12-31 21:00:00-05:00,263,247
4561438,2024-12-31 22:00:00-05:00,263,193


In [15]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# pip install confluent-kafka
# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-02 20:22:12,419 INFO: Initializing external client
2025-03-02 20:22:12,419 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:22:14,182 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [16]:
feature_store = project.get_feature_store()

In [17]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour"
)

In [19]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1212635/fs/1200268/fg/1403340


Uploading Dataframe: 100.00% |██████████| Rows 4560400/4560400 | Elapsed Time: 07:19 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [20]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 1780.65 MB


In [21]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4560400 entries, 0 to 4561439
Data columns (total 3 columns):
 #   Column              Dtype                     
---  ------              -----                     
 0   pickup_hour         datetime64[ns, US/Eastern]
 1   pickup_location_id  int16                     
 2   rides               int16                     
dtypes: datetime64[ns, US/Eastern](1), int16(2)
memory usage: 87.0 MB
